In [1]:
import ROOT
import numpy as np

def create_lorentz_vector(pt, eta, phi, m):
    lv = ROOT.TLorentzVector()
    lv.SetPtEtaPhiM(pt, eta, phi, m)
    return lv

def process_file_diphoton(file_path, tree_name):
    file_1 = ROOT.TFile(file_path, "READ")
    tree = file_1.Get(tree_name)

    diphoton_masses = []
    weights = []
    weight_central_values = []

    if not tree:
        print(f"Tree not found in file: {file_path}")
    else:
        for event in tree:
            if not hasattr(event, 'lead_pt') or not hasattr(event, 'lead_eta') or not hasattr(event, 'lead_phi') or not hasattr(event, 'sublead_pt') or not hasattr(event, 'sublead_eta') or not hasattr(event, 'sublead_phi'):
                continue

            lead_pt = event.lead_pt
            lead_eta = event.lead_eta
            lead_phi = event.lead_phi
            sublead_pt = event.sublead_pt
            sublead_eta = event.sublead_eta
            sublead_phi = event.sublead_phi
            weight = getattr(event, 'weight', 1)
            weight_central = getattr(event, 'weight_central', 1)

            photon_mass = 0.0

            lv1 = create_lorentz_vector(lead_pt, lead_eta, lead_phi, photon_mass)
            lv2 = create_lorentz_vector(sublead_pt, sublead_eta, sublead_phi, photon_mass)

            lv = lv1 + lv2
            invariant_mass = lv.M()

            diphoton_masses.append(invariant_mass)
            weights.append(weight)
            weight_central_values.append(weight_central)

    file_1.Close()
    return diphoton_masses, weights, weight_central_values

def process_file_dibjet(file_path, tree_name):
    file_1 = ROOT.TFile(file_path, "READ")
    tree = file_1.Get(tree_name)

    dibjet_masses = []
    weights = []
    weight_central_values = []

    if not tree:
        print(f"Tree not found in file: {file_path}")
    else:
        for event in tree:
            if not hasattr(event, 'lead_bjet_pt') or not hasattr(event, 'lead_bjet_eta') or not hasattr(event, 'lead_bjet_phi') or not hasattr(event, 'sublead_bjet_pt') or not hasattr(event, 'sublead_bjet_eta') or not hasattr(event, 'sublead_bjet_phi'):
                continue

            lead_bjet_pt = event.lead_bjet_pt
            lead_bjet_eta = event.lead_bjet_eta
            lead_bjet_phi = event.lead_bjet_phi
            lead_bjet_mass = event.lead_bjet_mass
            sublead_bjet_pt = event.sublead_bjet_pt
            sublead_bjet_eta = event.sublead_bjet_eta
            sublead_bjet_phi = event.sublead_bjet_phi
            sublead_bjet_mass = event.sublead_bjet_mass
            weight = getattr(event, 'weight', 1)
            weight_central = getattr(event, 'weight_central', 1)

            lv1 = create_lorentz_vector(lead_bjet_pt, lead_bjet_eta, lead_bjet_phi, lead_bjet_mass)
            lv2 = create_lorentz_vector(sublead_bjet_pt, sublead_bjet_eta, sublead_bjet_phi, sublead_bjet_mass)

            lv = lv1 + lv2
            invariant_mass = lv.M()

            dibjet_masses.append(invariant_mass)
            weights.append(weight)
            weight_central_values.append(weight_central)

    file_1.Close()
    return dibjet_masses, weights, weight_central_values


Welcome to JupyROOT 6.30/02


In [2]:

# Define the signal file path
signal_file = "../../output_root/GluGluToHH.root"
# Tree and variable names
tree_name = "DiphotonTree/data_125_13TeV_NOTAG"

# Branching ratio
BR_HToGG = 2.270E-03
BR_HTobb = 5.824E-01
BR_HTogg =  2.270E-03   #https://twiki.cern.ch/twiki/bin/view/LHCPhysics/CERNYellowReportPageBR

# Cross sections for backgrounds and signal
cross_sections = {
    "GGJets": 88.75,
    "GJetPt20To40": 242.5,
    "GJetPt40": 919.1,
    "GluGluHToGG": 52.23 * BR_HToGG,  
    "ttHToGG": 0.0013  ,
    "VBFHToGG": 0.00926 ,
    "VHToGG": 0.00545 ,
    "GluGluToHH":  34.43 * BR_HTobb * BR_HTogg * 2   # 0.1186     #0.1186   # ~34fb*BR(H-bb)*BR(H-gg)*2  = ~ 0.26% Checked
}



integrated_luminosities = {
    "Data_EraE": 5.8070,
    "Data_EraF": 17.7819,
    "Data_EraG": 3.0828
}

In [3]:
# Total integrated luminosity
total_integrated_luminosity = sum(integrated_luminosities.values())
print(f"Total integrated luminosity: {total_integrated_luminosity} fb")


Total integrated luminosity: 26.6717 fb


In [4]:
# Plot histograms
canvas = ROOT.TCanvas("canvas", "Invariant Mass plot", 800, 600)

# Adjust canvas to include ratio plot
pad1 = ROOT.TPad("pad1", "pad1", 0, 0.3, 1, 1.0)
pad1.SetBottomMargin(0.02)
pad1.SetTopMargin(0.1)
pad1.SetTicks(1, 1)
pad1.Draw()
pad1.cd()

# Draw the histograms on the same pad
hist_diphoton_mass.SetLineColor(ROOT.kBlue)
hist_diphoton_mass.SetLineWidth(2)
hist_diphoton_mass.Draw("HIST")

hist_dibjet_mass.SetLineColor(ROOT.kRed)
hist_dibjet_mass.SetLineWidth(2)
hist_dibjet_mass.Draw("SAME HIST")

hist_dibjet_mass.SetStats(0)
hist_diphoton_mass.SetStats(0)

# Add legend
legend = ROOT.TLegend(0.6, 0.6, 0.8, 0.8)
legend.AddEntry(hist_diphoton_mass, "Diphoton Mass", "l")
legend.AddEntry(hist_dibjet_mass, "Dibjet Mass", "l")
legend.Draw()

# Draw CMS text
cms_label = ROOT.TLatex()
cms_label.SetNDC()
cms_label.SetTextFont(61)
cms_label.SetTextSize(0.04)
cms_label.DrawLatex(0.1, 0.91, "CMS")

# Draw "Work in Progress"
work_label = ROOT.TLatex()
work_label.SetNDC()
work_label.SetTextFont(52)
work_label.SetTextSize(0.03)
work_label.DrawLatex(0.17, 0.91, "Work in Progress")

# Draw energy information
energy_label = ROOT.TLatex()
energy_label.SetNDC()
energy_label.SetTextFont(42)
energy_label.SetTextSize(0.03)
energy_label.DrawLatex(0.79, 0.91, "(13.6 TeV)")

# Set ticks on all sides with smaller size
hist_diphoton_mass.GetXaxis().SetTickSize(0.02)
hist_diphoton_mass.GetYaxis().SetTickSize(0.02)


# Save the canvas as a PDF file
# canvas.Print("signal_mass_comparison.pdf")

# Show the canvas
canvas.Draw()

NameError: name 'hist_diphoton_mass' is not defined

In [11]:
import ROOT

# Create histograms
hist_diphoton_mass = ROOT.TH1F("hist_diphoton_mass", "Diphoton Invariant Mass", 100, 100, 180)
hist_dibjet_mass = ROOT.TH1F("hist_dibjet_mass", "Dibjet Invariant Mass", 100, 0, 400)

def create_lorentz_vector(pt, eta, phi, m):
    lv = ROOT.TLorentzVector()
    lv.SetPtEtaPhiM(pt, eta, phi, m)
    return lv

def process_file_diphoton(file_path, tree_name):
    file_1 = ROOT.TFile(file_path, "READ")
    tree = file_1.Get(tree_name)

    if not tree:
        print(f"Tree not found in file: {file_path}")
        return

    for event in tree:
        if not hasattr(event, 'lead_pt') or not hasattr(event, 'lead_eta') or not hasattr(event, 'lead_phi') or not hasattr(event, 'sublead_pt') or not hasattr(event, 'sublead_eta') or not hasattr(event, 'sublead_phi'):
            continue

        lead_pt = event.lead_pt
        lead_eta = event.lead_eta
        lead_phi = event.lead_phi
        sublead_pt = event.sublead_pt
        sublead_eta = event.sublead_eta
        sublead_phi = event.sublead_phi

        lv1 = create_lorentz_vector(lead_pt, lead_eta, lead_phi, 0.0)  # Set photon mass to 0.0
        lv2 = create_lorentz_vector(sublead_pt, sublead_eta, sublead_phi, 0.0)

        lv = lv1 + lv2
        invariant_mass = lv.M()

        hist_diphoton_mass.Fill(invariant_mass)

    file_1.Close()

def process_file_dibjet(file_path, tree_name):
    file_1 = ROOT.TFile(file_path, "READ")
    tree = file_1.Get(tree_name)

    if not tree:
        print(f"Tree not found in file: {file_path}")
        return

    for event in tree:
        if not hasattr(event, 'lead_bjet_pt') or not hasattr(event, 'lead_bjet_eta') or not hasattr(event, 'lead_bjet_phi') or not hasattr(event, 'sublead_bjet_pt') or not hasattr(event, 'sublead_bjet_eta') or not hasattr(event, 'sublead_bjet_phi'):
            continue

        lead_bjet_pt = event.lead_bjet_pt
        lead_bjet_eta = event.lead_bjet_eta
        lead_bjet_phi = event.lead_bjet_phi
        lead_bjet_mass = event.lead_bjet_mass
        sublead_bjet_pt = event.sublead_bjet_pt
        sublead_bjet_eta = event.sublead_bjet_eta
        sublead_bjet_phi = event.sublead_bjet_phi
        sublead_bjet_mass = event.sublead_bjet_mass

        lv1 = create_lorentz_vector(lead_bjet_pt, lead_bjet_eta, lead_bjet_phi, lead_bjet_mass)
        lv2 = create_lorentz_vector(sublead_bjet_pt, sublead_bjet_eta, sublead_bjet_phi, sublead_bjet_mass)

        lv = lv1 + lv2
        invariant_mass = lv.M()

        hist_dibjet_mass.Fill(invariant_mass)

    file_1.Close()

# Define the signal file path
signal_file = "../../output_root/GluGluToHH.root"
# Tree and variable names
tree_name_diphoton = "DiphotonTree/data_125_13TeV_NOTAG"
tree_name_dibjet = "DiphotonTree/data_125_13TeV_NOTAG"

# Process files
process_file_diphoton(signal_file, tree_name_diphoton)
process_file_dibjet(signal_file, tree_name_dibjet)

# Plot histograms
canvas = ROOT.TCanvas("canvas", "Invariant Mass plot", 800, 600)

# Draw the histograms on the same pad
hist_diphoton_mass.SetLineColor(ROOT.kBlue)
hist_diphoton_mass.SetLineWidth(2)
hist_diphoton_mass.SetStats(0)  # Remove statistics box
hist_diphoton_mass.GetXaxis().SetTitle(" (GeV)")
hist_diphoton_mass.GetYaxis().SetTitle("Events")
hist_diphoton_mass.GetYaxis().SetTitleOffset(1.4)
hist_diphoton_mass.GetYaxis().SetLabelSize(0.03)
hist_diphoton_mass.Draw("HIST")

hist_dibjet_mass.SetLineColor(ROOT.kRed)
hist_dibjet_mass.SetLineWidth(2)
hist_dibjet_mass.SetStats(0)  # Remove statistics box
hist_dibjet_mass.Draw("SAME HIST")

# Add legend
legend = ROOT.TLegend(0.6, 0.6, 0.8, 0.8)
legend.AddEntry(hist_diphoton_mass, "Diphoton Mass", "l")
legend.AddEntry(hist_dibjet_mass, "Dibjet Mass", "l")
legend.Draw()

# Draw CMS text
cms_label = ROOT.TLatex()
cms_label.SetNDC()
cms_label.SetTextFont(61)
cms_label.SetTextSize(0.04)
cms_label.DrawLatex(0.1, 0.91, "CMS")

# Draw "Work in Progress"
work_label = ROOT.TLatex()
work_label.SetNDC()
work_label.SetTextFont(52)
work_label.SetTextSize(0.03)
work_label.DrawLatex(0.17, 0.91, "Work in Progress")

# Draw energy information
energy_label = ROOT.TLatex()
energy_label.SetNDC()
energy_label.SetTextFont(42)
energy_label.SetTextSize(0.03)
energy_label.DrawLatex(0.81, 0.91, "(13.6 TeV)")


# Save the canvas as a PDF file
canvas.Print("/afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/signal_mass_comparison.pdf")

# Show the canvas
canvas.Draw()


Warning in <TROOT::Append>: Replacing existing TH1: hist_diphoton_mass (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: hist_dibjet_mass (Potential memory leak).
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas
Info in <TCanvas::Print>: pdf file /afs/cern.ch/user/s/sraj/sraj/www/CUA/HH-bbgg/signal_mass_comparison.pdf has been created
